In [113]:
import os
import pandas as pd
from unidecode import unidecode

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [114]:
def remover_acentos(texto):
    return unidecode(texto)

In [115]:
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA

# IMPORTS

In [116]:
league = {'brasileirao_a': ['Brasileirao'],
          'premier_league': ['Premier_League'],}


liga = league['brasileirao_a'][0]

In [117]:
historico = pd.read_excel(f'datasets/{liga}/match_history/historico-2015a2023(naotratada).xlsx')

In [118]:
historico.drop(historico.columns[0], axis=1, inplace=True)

# HISTORICO

### Data cleaning

In [119]:
historico.drop(
  ['notes','captain', 'formation', 'referee', 'match report', 'attendance', 'time'], 
  axis= 'columns', inplace= True)

In [120]:
historico[['gf', 'ga']] = historico[['gf', 'ga']].astype(int)
historico['season'] = historico['season'].astype(int)
historico['round'] = historico['round'].str.split(' ', expand= True)[1].astype(int)

In [121]:
historico['opponent'] = historico['opponent'].apply(remover_acentos)
historico['opponent'] = [linha.lower() for linha in historico['opponent']]
historico['opponent'] = historico['opponent'].str.replace(' ', '_')
historico['opponent'] = historico['opponent'].str.replace('(', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')

# Especifico para o brasileirão
historico['opponent'] = historico['opponent'].str.replace('atletico', 'atl')
historico['team'] = historico['team'].str.replace('athletico', 'ath')
historico['team'] = historico['team'].str.replace('atletico', 'atl')

In [122]:
historico.head()

,date,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,sh,sot,season,team
0,2023-04-15,Série A,1,Sat,Home,W,2,1,sao_paulo,0.9,2.0,33.0,12.0,3.0,2023,botafogo_rj
1,2023-04-24,Série A,2,Mon,Away,W,2,1,bahia,0.2,0.9,42.0,5.0,3.0,2023,botafogo_rj
2,2023-04-30,Série A,3,Sun,Away,W,3,2,flamengo,1.9,3.1,26.0,11.0,5.0,2023,botafogo_rj
3,2023-05-07,Série A,4,Sun,Home,W,2,0,atl_mineiro,2.0,0.5,39.0,22.0,7.0,2023,botafogo_rj
4,2023-05-11,Série A,5,Thu,Home,W,3,0,corinthians,2.6,0.6,53.0,15.0,5.0,2023,botafogo_rj


### Divisão do DF

Casa

In [123]:
historico_casa = historico[historico['venue'] == 'Home']
historico_casa.to_excel(f'datasets/{liga}/match_history/home_only/historico-2015a2023.xlsx', index=False)

In [128]:
for season in historico_casa['season'].unique():
  if not os.path.exists(f'dataset/{liga}/match_history/home_only/season-{season}'):
    os.makedirs(f'dataset/{liga}/match_history/home_only/season-{season}')
  df = historico_casa[historico_casa['season'] == season]
  df.to_excel(f'dataset/{liga}/match_history/home_only/season-{season}/historico.xlsx', index= False)

Por Time

In [ ]:
for time in historico['team']:
  df = historico[historico['team'] == time]
  df.to_excel(f'dataset/{liga}/match_history/teams/{time}.xlsx', index= False)

In [7]:
data_ultimo_jogo = historico['Round'].max()
historico.to_excel(f'dados/historico/historico-rodada_{data_ultimo_jogo}.xlsx', sheet_name='historico', index=False)


In [ ]:
resultados_rodada = historico_casa[historico_casa['Round'] == rodada]
resultados_rodada.to_excel(f'dados/historico/resultados/rodada-{rodada}.xlsx')

# TABELA

In [ ]:
tabela_df.drop(['Attendance', 'Top Team Scorer', 'Goalkeeper', 'Notes'], axis= 'columns', inplace= True)
tabela_df['xGConv'] = tabela_df['GF'] - tabela_df['xG']   #Conversão de xG
tabela_df['xGAConv'] = tabela_df['GA'] - tabela_df['xGA']   #Conversão de xGA
tabela_df['GF/MP'] = tabela_df['GF']/tabela_df['MP']    #Gols feitos por partida jogada
tabela_df['GA/MP'] = tabela_df['GA']/tabela_df['MP']    #Gols recebidos por partida jogada
tabela_df['Squad'] = tabela_df['Squad'].apply(remover_acentos)

In [ ]:
#ATT Rating = (Gols Feitos no campeonato/Partidas Jogadas)/gols_liga
#DEF Rating = (Gols Sofridos no campeonato/Partidas Jogadas)/gols_liga
#EXPECTED GOALS = ATTxDEFxAVGLeague

gols_feitos = tabela_df['GF']/tabela_df['MP']
gols_tomados = tabela_df['GA']/tabela_df['MP']
gols_liga = tabela_df['GF']/tabela_df['MP'].mean()

att_rating = gols_feitos/gols_liga
def_rating = gols_tomados/gols_liga

tabela_df['expG'] = att_rating*def_rating*gols_liga

#xG e Gols, fora e dentro de casa
for time in tabela_df['Squad']:
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_casa'] = historico_casa[historico_casa['Equipe'] == time]['xG'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_fora'] = historico_casa[historico_casa['Opponent'] == time]['xGA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_total'] = tabela_df['xG_casa'] + tabela_df['xG_fora']

  tabela_df.loc[tabela_df['Squad'] == time, 'GF_casa'] = historico_casa[historico_casa['Equipe'] == time]['GF'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GF_fora'] = historico_casa[historico_casa['Opponent'] == time]['GA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GF_total'] = tabela_df['GF_casa'] + tabela_df['GF_fora']

  tabela_df.loc[tabela_df['Squad'] == time, 'GA_casa'] = historico_casa[historico_casa['Equipe'] == time]['GA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GA_fora'] = historico_casa[historico_casa['Opponent'] == time]['GF'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GA_total'] = tabela_df['GA_casa'] + tabela_df['GA_fora']

  tabela_df.loc[tabela_df['Squad'] == time, 'xG_sofrido_casa'] = historico_casa[historico_casa['Equipe'] == time]['xGA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_sofrido_fora'] = historico_casa[historico_casa['Opponent'] == time]['xG'].mean()
  
  tabela_df.loc[tabela_df['Squad'] == time, 'Sh_casa'] = historico_casa[historico_casa['Equipe'] == time]['Sh'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'SoT_casa'] = historico_casa[historico_casa['Equipe'] == time]['SoT'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'Saves_casa'] = historico_casa[historico_casa['Equipe'] == time]['Saves'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GCA_casa'] = historico_casa[historico_casa['Equipe'] == time]['GCA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'Tklw_casa'] = historico_casa[historico_casa['Equipe'] == time]['TklW'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'Sh_fora'] = historico_casa[historico_casa['Opponent'] == time]['Sh'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'SoT_fora'] = historico_casa[historico_casa['Opponent'] == time]['SoT'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'Saves_fora'] = historico_casa[historico_casa['Opponent'] == time]['Saves'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GCA_fora'] = historico_casa[historico_casa['Opponent'] == time]['GCA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'Tklw_fora'] = historico_casa[historico_casa['Opponent'] == time]['TklW'].mean()
    

#Ultimos 5 jogos, xG e Gols, fora e dentro de casa
historico_5jogos = historico_casa[historico_casa['Round'] > rodada-5]

for time in tabela_df['Squad']:
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['xG'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['xGA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_5total'] = tabela_df['xG_casa'] + tabela_df['xG_fora']
  tabela_df.loc[tabela_df['Squad'] == time, 'GF_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['GF'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GF_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['GA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GF_5total'] = tabela_df['GF_5casa'] + tabela_df['GF_5fora']
  tabela_df.loc[tabela_df['Squad'] == time, 'GA_5casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['GA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GA_5fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['GF'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'GA_5total'] = tabela_df['GA_5casa'] + tabela_df['GA_5fora']
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_5sofrido_casa'] = historico_5jogos[historico_5jogos['Equipe'] == time]['xGA'].mean()
  tabela_df.loc[tabela_df['Squad'] == time, 'xG_5sofrido_fora'] = historico_5jogos[historico_5jogos['Opponent'] == time]['xG'].mean()

In [ ]:
tabela_df.to_excel(f'dados/tabela/tabela-rodada_{rodada}.xlsx', sheet_name='tabela', index=False)

# RODADAS

In [ ]:
rodadas_df = rodadas_df.loc[rodadas_df['Home'].notna()]
rodadas_df = rodadas_df.loc[rodadas_df['Away'].notna()]
rodadas_df['Home'] = rodadas_df['Home'].astype(str).apply(remover_acentos)
rodadas_df['Away'] = rodadas_df['Away'].astype(str).apply(remover_acentos)

substitutions = {
    'Botafogo-RJ': 'Botafogo (RJ)',
    'Gremio': 'Gremio',
    'Athletico-Paranaense': 'Ath Paranaense',
    'Atletico-Mineiro': 'Atletico Mineiro',
    'Sao-Paulo': 'Sao Paulo',
    'Vasco-da-Gama': 'Vasco da Gama',
    'America-MG': 'America (MG)'
}

for key, value in substitutions.items():
    rodadas_df['Home'] = rodadas_df['Home'].str.replace(unidecode(key), value, regex=True)
    rodadas_df['Away'] = rodadas_df['Away'].str.replace(unidecode(key), value, regex=True)

rodadas_df.drop(['Time', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'], axis= 1, inplace= True)
rodadas_df[['gols_casa', 'gols_fora']] = rodadas_df['Score'].str.split('–', expand=True)
rodadas_df.drop(['Score'], axis= 1, inplace= True)
rodadas_df.insert(5, 'gols_casa', rodadas_df.pop('gols_casa'))
rodadas_df.insert(6, 'gols_fora', rodadas_df.pop('gols_fora'))
rodadas_df = rodadas_df.rename(columns={'xG.1': 'xGFora', 'xG': 'xGCasa'})
rodadas_df['gols_casa'] = rodadas_df['gols_casa'].apply(convert_to_int)
rodadas_df['gols_fora'] = rodadas_df['gols_fora'].apply(convert_to_int)

In [ ]:
rodadas_df.to_excel(f'dados/rodadas/todas_rodadas.xlsx', sheet_name='tabela', index=False)

Rodadas futuras

In [ ]:
rodadas_futuras = rodadas_df[rodadas_df['gols_fora'].isna()]
rodadas_futuras.to_excel(f'dados/rodadas/rodadas_futuras-rodada-{rodada+1}.xlsx', sheet_name='tabela', index=False)

In [ ]:
rodada_futura = rodadas_futuras[rodadas_futuras['Wk'] == rodada+1]
rodada_futura.to_excel(f'dados/rodadas/rodada-{rodada+1}.xlsx', sheet_name='tabela', index=False)

# Tabela em casa

In [ ]:
for time in tabela_df['Squad']:
  pontos = 0
  ldw = pd.DataFrame(historico_casa[historico_casa['Equipe'] == time]['Result'].value_counts()).reset_index()
  for i in range(0, len(ldw)):
    if ldw['index'][i] == 'W':
      pontos += ldw['Result'][i]*3
    if ldw['index'][i] == 'D':
      pontos += ldw['Result'][i]
  tabela_df.loc[tabela_df['Squad'] == time, 'pontos_casa'] = pontos  

tabela_casa = tabela_df.sort_values(by= 'pontos_casa',ascending=	False)
tabela_casa['Rk_casa'] = range(1, 21)

In [ ]:
tabela_casa.to_excel(f'dados/tabela/tabela_casa-rodada_{rodada}.xlsx')